# Demand Analysis

In [ ]:
# Import libraries
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Load your sales data
sales_df = pd.read_csv('clothing_reviews.csv')  # Example: columns ['Order_ID', 'Product_ID', 'Quantity', 'Date']

sales_df.head()


In [ ]:
products_df = pd.read_csv('clothing_description.csv')
products_df.head()

# 1. Best-Selling / Trending Products (Customer page)

In [ ]:
import pandas as pd

# Load the datasets
products_df = pd.read_csv('clothing_description.csv')
reviews_df = pd.read_csv('clothing_reviews.csv')

# Step 1: Find Best-Selling Products
best_sellers = reviews_df.groupby('product_id')['Quantity'].sum().sort_values(ascending=False)

# Step 2: Top 10 Trending Product IDs
top_trending_ids = best_sellers.head(10).index.tolist()

# Step 3: Count number of non-empty reviews per product
review_counts = reviews_df[reviews_df['Review Text'].notnull()].groupby('product_id')['Review Text'].count().reset_index()
review_counts.rename(columns={'Review Text': 'num_reviews'}, inplace=True)

# Step 4: Get product details for trending products
trending_products = products_df[products_df['product_id'].isin(top_trending_ids)]

# Step 5: Merge with review counts
trending_products = trending_products.merge(review_counts, on='product_id', how='left')
trending_products['num_reviews'].fillna(0, inplace=True)  # If no reviews, set to 0
trending_products['num_reviews'] = trending_products['num_reviews'].astype(int)

# Optional: Order by sales volume (same as best_sellers)
trending_products['total_quantity'] = trending_products['product_id'].map(best_sellers)
trending_products = trending_products.sort_values(by='total_quantity', ascending=False)

# Step 6: Show Trending Products
print("🔥 Trending Products (Top 10 by Sales) + Number of Reviews:")
print(trending_products[['product_id', 'product_name', 'product_category', 'total_quantity', 'num_reviews']].to_string(index=False))


# 2. Admin Page – Demand Forecasting (Next 6 months)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings
warnings.filterwarnings('ignore')

# Remove rows with missing key values
sales_df = sales_df.dropna(subset=['product_id', 'Quantity', 'Purchase Date'])

# Correct datetime format
sales_df['Purchase Date'] = pd.to_datetime(sales_df['Purchase Date'], format='%d-%m-%Y %H:%M')

# Set datetime as index
sales_df.set_index('Purchase Date', inplace=True)

# Aggregate monthly sales
monthly_total_sales = sales_df.resample('M')['Quantity'].sum()

# Drop the last month if it's not a full month (based on last date)
last_date = sales_df.index.max()
if last_date.day < 28:  # Likely not a full month
    monthly_total_sales = monthly_total_sales[:-1]

# Interpolate any 0s that may have been caused by missing data
monthly_total_sales = monthly_total_sales.replace(0, pd.NA)
monthly_total_sales = monthly_total_sales.interpolate()

# Fit Exponential Smoothing model (additive trend & seasonality)
model = ExponentialSmoothing(monthly_total_sales, trend='add', seasonal='add', seasonal_periods=12).fit()

# Forecast next 6 months
forecast = model.forecast(6)

# --- Key Change: Merge historical and forecast for smoother plot ---
full_series = pd.concat([monthly_total_sales, forecast])

# Plot
plt.figure(figsize=(12,6))
plt.plot(full_series, label='Historical + Forecasted Sales', color='blue')  # Single continuous line
plt.axvline(x=monthly_total_sales.index[-1], color='orange', linestyle='--', label='Forecast Start')  # Mark forecast start
plt.title('Demand Forecast - Next 6 Months')
plt.xlabel('Month')
plt.ylabel('Total Quantity Sold')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Print forecasted values
print("📈 Forecasted Sales for Next 6 Months:")
print(forecast)


# Predict high-demand products and alert admin to stock them.

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Load files
desc_df = pd.read_csv('clothing_description.csv')
reviews_df = pd.read_csv('clothing_reviews.csv')

# Convert 'Purchase Date' to datetime
reviews_df['Purchase Date'] = pd.to_datetime(reviews_df['Purchase Date'], dayfirst=True)

# Remove missing or invalid product IDs/quantities
reviews_df.dropna(subset=['product_id', 'Quantity', 'Purchase Date'], inplace=True)


In [ ]:
# Set date as index for resampling
reviews_df.set_index('Purchase Date', inplace=True)

# Group by product and month
monthly_sales = reviews_df.groupby(['product_id', pd.Grouper(freq='M')])['Quantity'].sum().reset_index()


In [ ]:
alerts = []

for product_id in monthly_sales['product_id'].unique():
    product_data = monthly_sales[monthly_sales['product_id'] == product_id].copy()
    product_data.set_index('Purchase Date', inplace=True)
    ts = product_data['Quantity']

    if len(ts) < 6:
        continue

    try:
        sp = min(12, len(ts))  # Use shorter seasonal period if needed
        model = ExponentialSmoothing(ts, trend='add', seasonal='add', seasonal_periods=sp).fit()
        forecast = model.forecast(1)
        predicted_demand = forecast.iloc[0]

        print(f"Product {product_id} forecasted demand: {predicted_demand:.2f}")  # Debug

        if predicted_demand >= 10:  # Lower threshold for testing
            alerts.append((product_id, round(predicted_demand, 2)))

    except Exception as e:
        print(f"Skipped {product_id} due to error: {e}")


In [ ]:
# Merge forecasted products with product details
alerts_df = pd.DataFrame(alerts, columns=['product_id', 'forecasted_quantity'])
final_alerts = pd.merge(alerts_df, desc_df, on='product_id', how='left')

# Show alert
print("⚠️ High-Demand Products - Stock Alert for Next Month ⚠️")
print(final_alerts[['product_id', 'product_name', 'product_category', 'forecasted_quantity']].sort_values(by='forecasted_quantity', ascending=False).to_string(index=False))


# 3. Admin Page - Low Demand Products (Discount/Promotion)

In [ ]:
df = pd.read_csv("clothing_reviews.csv")

# Convert 'Purchase Date' to datetime
df['Purchase Date'] = pd.to_datetime(df['Purchase Date'], dayfirst=True)

# Determine the latest date in the dataset
latest_date = df['Purchase Date'].max()

# Filter data for the last 3 months from the latest purchase date
three_months_ago = latest_date - pd.DateOffset(months=3)
recent_df = df[df['Purchase Date'] >= three_months_ago]

# Group by product_id and product_name, summing the total quantity sold
recent_demand = recent_df.groupby(['product_id', 'product_name'])['Quantity'].sum().reset_index()

# Sort by quantity to find low-demand products
low_recent_demand = recent_demand.sort_values(by='Quantity').reset_index(drop=True)

# Display low-demand products in the last 3 months
low_recent_demand.head(10)

In [ ]:
# Define discount strategy function
def suggest_discount(qty):
    if qty <= 1:
        return "30% OFF"
    elif qty <= 2:
        return "20% OFF"
    elif qty <= 3:
        return "10% OFF"
    else:
        return "No Discount"

# Apply the discount suggestion
low_recent_demand['Suggested Discount'] = low_recent_demand['Quantity'].apply(suggest_discount)

# Filter to show only products that should be discounted
discount_suggestions = low_recent_demand[low_recent_demand['Suggested Discount'] != "No Discount"]

# Display final suggested discount list
print(discount_suggestions)
